# Seção 3: Otimização do Tempo de Execução

___

# Imports  Para a Aula

In [ ]:
import numpy as np

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from tree import BinaryTree, extract_tree_from_model, print_tree

___

# Tutorial

## Motivação: Contagem Simples

### Criando dados aleatórios

In [ ]:
np.random.seed(123456789)
X = np.random.randint(10, 35, int(1e7))
X.shape, X[:10]

### Função "Mágica" _%time_

In [ ]:
%%time
count1 = 0
for x in X:
    if x > 20:
        count1 += 1

In [ ]:
%%time
count2 = (X > 20).sum()

In [ ]:
assert count1 == count2

### Função "Mágica" _%timeit_

In [ ]:
%%timeit
count1 = 0
for x in X:
    if x > 20:
        count1 += 1

In [ ]:
%%timeit
count2 = (X > 20).sum()

In [ ]:
assert count1 == count2

## Motivação: Transformação Linear

### Funções Auxiliares

In [ ]:
def matrix_dot_product(m1, m2):    
    assert (m1.shape[1] == m2.shape[0])
    P, Q = m1.shape
    Q, R = m2.shape
    ans = np.zeros((P, R))
    for p in range(P):
        for r in range(R):
            for q in range(Q):
                ans[p, r] += m1[p, q] * m2[q, r]
    return ans

In [ ]:
def matrix_add(m1, m2):
    assert (m1.shape == m2.shape)
    P, Q = m1.shape
    ans = np.zeros((P, Q))
    for p in range(P):
        for q in range(Q):
            ans[p, q] += m1[p, q] + m2[p, q]
    return ans

### Criando dados aleatórios

In [ ]:
np.random.seed(123456789)
P = 500
Q = 50
R = 1
A = np.random.randint(-10, 35, (P, Q))
X = np.random.randn(Q, R) * 10 + 3
B = np.random.randn(P, R) * 3 + 10
A.shape, X.shape, B.shape

### Função "Mágica" _%time_

In [ ]:
%%time
Y1 = matrix_add(matrix_dot_product(A, X), B)

In [ ]:
%%time
Y2 = np.add(np.dot(A, X), B)

In [ ]:
assert sum((Y1 - Y2) ** 2) ** .5 < 1e-10

### Função "Mágica" _%timeit_

In [ ]:
%%timeit
Y1 = matrix_add(matrix_dot_product(A, X), B)

In [ ]:
%%timeit
Y2 = np.add(np.dot(A, X), B)

In [ ]:
assert sum((Y1 - Y2) ** 2) ** .5 < 1e-10

___

# Desafio

## Objetivo:

Construir uma classe de Árvore de Decisão derivada de uma classe de Árvore Binária, disponibilizada aos alunos no arquivo ``tree.py`` deste módulo. 

Não é necessário implementar o **treinamento** (ou métoro **fit**) da Árvode de Decisão; tampouco o método de predição de probabilidade (**predict_proba**). Todos os parâmetros da Árvore de Decisão serão extraídos de um modelo de `DecisionTreeClassifier` treinado com o dataset `Iris`.

Essa classe deve extender a classe disponibilizada `BinaryTree`. Cada instância dessa classe representa **apenas um nó** de uma árvore binária; este nó está conectado aos outros nós através dos seguintes atributos:
- `parent`
- `left_child`
- `right_child`

A classe `BinaryTree` não possui um método `predict`; este deve ser implementado para funcionar da seguinte forma:
1. Se for um **nó folha**, retorna a predição para o **nó pai**;
2. Se for um **nó de decisão**, deve processar a entrada `X` e chamar o método `predict` do próximo nó filho.

O método `predict` deve receber um array `X` contendo uma massa de dados sobre os quais serão feitas as predições, que serão retornadas no array `y_pred`.

## Árvore de Decisão

### Treinando uma Árvore de Decisão para o Problema de Classificação Iris

Referências:
- [Problema de Classificação: Iris](https://en.wikipedia.org/wiki/Iris_flower_data_set)
- [Árvore de Decisão para Classificação](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

In [ ]:
iris = load_iris()

In [ ]:
model = DecisionTreeClassifier(max_depth=3, random_state=123456789)

In [ ]:
model.fit(X=iris['data'], y=iris['target'])

### Testando a Predição da Árvore de Decisão Treinada

In [ ]:
""" predições """
y_pred = model.predict(iris['data'])

In [ ]:
""" targets / tabela verdade """
y_true = iris['target']

In [ ]:
""" Acurácia do Modelo """
(y_pred == y_true).mean()

### Parâmetros da Árvore de Decisão Treinada:

In [ ]:
tree = extract_tree_from_model(model, BinaryTree)
print_tree(tree)

## Parte 1: Implementar a solução usando apenas loops

### Solução:

In [ ]:
""" Escreva a a Solução Aqui """
class LoopDecisionTree(BinaryTree):
    
    def __init__(self, threshold=None, feature=None, decision=None, *args, **kwargs):
        super(LoopDecisionTree, self).__init__(*args, **kwargs)
        pass
    
    def predict(self, X):        
        return None

### Avaliação da Solução

In [ ]:
""" Construindo a árvore de decisão """
# classe de Árvore de Decisão
DTClass = LoopDecisionTree

# Criando os Nós da Árvore
n0 = DTClass(id=0, feature=3, threshold=0.80000001192092896)
n1 = DTClass(id=1, decision=0)
n2 = DTClass(id=2, feature=3, threshold=1.75)
n3 = DTClass(id=3, feature=2, threshold=4.9499998092651367)
n4 = DTClass(id=4, decision=1)
n5 = DTClass(id=5, decision=2)
n6 = DTClass(id=6, feature=2, threshold=4.8500003814697266)
n7 = DTClass(id=7, decision=2)
n8 = DTClass(id=8, decision=2)

# Construindo a Árvore
n0.append_left_child(n1)
n0.append_right_child(n2)
n2.append_left_child(n3)
n2.append_right_child(n6)
n3.append_left_child(n4)
n3.append_right_child(n5)
n6.append_left_child(n7)
n6.append_right_child(n8)

decision_tree = n0

In [ ]:
assert (model.predict(iris['data']) == decision_tree.predict(iris['data'])).all()

## Parte 2: Implementar a solução usando _numpy_

### Solução:

In [ ]:
""" Escreva a a Solução Aqui """
class VectorDecisionTree(BinaryTree):    
        
    def __init__(self, threshold=None, feature=None, decision=None, *args, **kwargs):
        super(VectorDecisionTree, self).__init__(*args, **kwargs)
        pass
    
    def predict(self, X):
        return None

### Avaliação da Solução

In [ ]:
""" Construindo a árvore de decisão """
# classe de Árvore de Decisão
DTClass = VectorDecisionTree

# Criando os Nós da Árvore
n0 = DTClass(id=0, feature=3, threshold=0.80000001192092896)
n1 = DTClass(id=1, decision=0)
n2 = DTClass(id=2, feature=3, threshold=1.75)
n3 = DTClass(id=3, feature=2, threshold=4.9499998092651367)
n4 = DTClass(id=4, decision=1)
n5 = DTClass(id=5, decision=2)
n6 = DTClass(id=6, feature=2, threshold=4.8500003814697266)
n7 = DTClass(id=7, decision=2)
n8 = DTClass(id=8, decision=2)

# Construindo a Árvore
n0.append_left_child(n1)
n0.append_right_child(n2)
n2.append_left_child(n3)
n2.append_right_child(n6)
n3.append_left_child(n4)
n3.append_right_child(n5)
n6.append_left_child(n7)
n6.append_right_child(n8)

decision_tree = n0

In [ ]:
assert (model.predict(iris['data']) == decision_tree.predict(iris['data'])).all()

## Parte 3: Avaliar a diferença de velocidade

### Solução:

In [ ]:
""" Escreva a a Solução Aqui """